In [10]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import trange
from utils import SphericalNODataset, get_cr_dirs
from io import BytesIO
import imageio.v2 as imageio

In [2]:
data_path = '/Users/reza/Career/DMLab/SURROGATE/Data/psi_web_sample/train'

In [21]:
cr_dirs = get_cr_dirs(data_path)
print(len(cr_dirs))
dataset = SphericalNODataset(data_path, cr_dirs)
climatology = dataset.climatology
print(climatology.shape)

25


Loading simulations: 100%|██████████| 25/25 [00:00<00:00, 72.76it/s]


torch.Size([139, 111, 128])


In [23]:
instance_1 = dataset[0]['y']
instance_2 = dataset[1]['y']

In [28]:
frames = []
for i in trange(139):
    
    y_1 = instance_1[i, :, :]
    y_2 = instance_2[i, :, :]
    clim = climatology[i, :, :]
    
    vmin = min(y_1.min(), y_2.min(), climatology.min())
    vmax = max(y_1.max(), y_2.max(), climatology.max())
    fig, axes = plt.subplots(1, 3, figsize=(12, 6))
    cmap = "gnuplot"

    # # Plot first subplot
    im1 = axes[0].imshow(y_1, cmap=cmap, vmin=vmin, vmax=vmax)
    axes[0].set_title(f"vr002$_i$ at {i+1}")

    # Plot second subplot
    im3 = axes[1].imshow(y_2, cmap=cmap, vmin=vmin, vmax=vmax)
    axes[1].set_title(f"vr002$_j$ at {i+1}")

    # Plot second subplot
    im3 = axes[2].imshow(clim, cmap=cmap, vmin=vmin, vmax=vmax)
    axes[2].set_title(f"climatology at {i+1}")

    cbar = fig.colorbar(im1, ax=axes, orientation="horizontal", fraction=0.1, pad=0.02)
    
    fig.suptitle("Climatology from 25 cubes", fontsize=16)

    # plt.tight_layout()
    buf = BytesIO()
    plt.savefig(buf, format="png")
    frames.append(imageio.imread(buf))
    plt.close()

100%|██████████| 139/139 [00:22<00:00,  6.08it/s]


In [29]:
output_filename = f"climatology.gif"
fps = 10  # Adjust frames per second as needed

# Create video directly from frames in memory
with imageio.get_writer(output_filename, fps=fps, loop=0) as writer:
    for frame in frames:
        writer.append_data(frame)